<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-08-06 11:48:54
-------------------
qualified stocks: 92
with latest results: 28
still star stocks: 18
-------------------
Initial Investment:  1.20 C
CY Investment:  1.47 C
Reserve:  3.53 L
Current:  1.35 C
-------------------
Today PnL: -1.42 L (-1.05%)
Current PnL: -20.30 L (-13.81%)
CY Booked + Current PnL: -12.03 L (-8.18%)
-------------------
Total profit:  2.00 L
Total loss:  -22.30 L
-------------------
Total Booked + Current PnL: 14.95 L (12.45%)
Total Booked PnL: 35.25 L (29.36%)
Curr Year Booked PnL: 8.27 L (6.15%)
Prev Year Booked PnL: 26.98 L (22.48%)
Est FTT:  2.13 C
Est FTT PnL: 78.57 L (58.38%)
Deployed:  1.20 C
Current:  1.35 C
CAGR/XIRR %: 7.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110136.0,5824.0,11707.0,0.18,5.58,10.63,16.81,10.0,0.50,0.84,22.53,X5K,ATH,METALS
67,SBIN,760.30,863.00,-12.46,M-LC,3.94,212225.0,10746.0,16469.0,0.03,5.33,7.76,13.51,62.0,0.65,1.62,18.60,XY25,NTT,BANKS
82,TTKPRESTIG,769.29,770.00,-9.57,H-SC,11.65,82766.0,-18011.0,18101.0,0.51,-17.87,21.87,0.09,145.0,-1.00,0.63,8.74,OX40N,NTT,DURABLES
71,SIS,477.00,477.00,2230.05,H-SC,6.98,62688.0,-18402.0,18405.0,-1.44,-22.69,29.36,0.00,157.0,-1.00,0.48,27.05,OX40N,NTT,MISC
27,DIXON,14367.00,18940.15,-28.24,H-MC,6.61,132152.0,17216.0,19373.0,-1.96,14.98,14.66,31.83,85.0,0.89,1.01,48.73,X40N,ATH,IT


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BERGEPAINT,561.33,680.0,-19.49,H-MC,9.9,228076.0,737.0,47326.0,-1.5,0.32,20.75,21.14,107.0,0.02,1.74,28.56,XY24,NTT,PAINTS


In [11]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,CIPLA,1492.70,1795.0,-23.18,H-LC,1.78,202527.0,-1973.0,43381.0,-1.43,-0.96,21.42,20.25,9.0,-0.05,1.55,7.68,X40N,BTT,PHARMA
5,ASIANPAINT,2961.56,4250.0,-14.80,H-LC,2.63,169184.0,-32202.0,119816.0,2.10,-15.99,70.82,43.51,19.0,-0.27,1.29,17.77,X40,BTT,PAINTS
53,LTIM,5564.16,7230.2,-5.08,H-LC,3.22,182520.0,-17790.0,77772.0,-1.20,-8.88,42.61,29.94,24.0,-0.23,1.39,26.09,X200,ATH,IT
10,BAJAJHFL,122.26,152.0,-10.62,H-LC,3.76,185484.0,-15022.0,63788.0,-0.99,-7.49,34.39,24.33,30.0,-0.24,1.42,6.96,X40N,BTT,FINANCE
57,NESTLEIND,2268.60,2755.0,-17.04,H-LC,4.15,263133.0,-2293.0,59205.0,-0.81,-0.86,22.50,21.44,26.0,-0.04,2.01,5.72,XY25,NTT,FMCG


In [12]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,2961.56,4250.00,-14.80,H-LC,2.63,169184.0,-32202.0,119816.0,2.10,-15.99,70.82,43.51,19.0,-0.27,1.29,17.77,X40,BTT,PAINTS
22,COALINDIA,406.65,504.72,-1.52,L-LC,2.20,138380.0,-12080.0,48364.0,1.28,-8.03,34.95,24.12,169.0,-0.25,1.06,7.14,XY25,ATH,MINING
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.39,133491.0,10155.0,123613.0,1.28,8.23,92.60,108.45,118.0,0.08,1.02,32.43,AR,ATH,MISC
82,TTKPRESTIG,769.29,770.00,-9.57,H-SC,11.65,82766.0,-18011.0,18101.0,0.51,-17.87,21.87,0.09,145.0,-1.00,0.63,8.74,OX40N,NTT,DURABLES
54,MASFIN,326.60,399.50,-20.05,H-SC,16.30,91065.0,-6915.0,28786.0,0.50,-7.06,31.61,22.32,151.0,-0.24,0.69,32.08,XR,ATH,FINANCE


In [13]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,RAJESHEXPO,517.65,518.00,1661.36,L-SC,3.43,48071.0,-89106.0,89201.0,-4.08,-64.96,185.56,0.07,266.0,-1.00,0.37,19.55,OX40N,NTT,JEWELLERY
75,SYMPHONY,1306.42,1306.00,-39.43,M-SC,9.43,129087.0,-42054.0,42005.0,-3.41,-24.57,32.54,-0.03,197.0,-1.00,0.99,0.00,OX40N,NTT,DURABLES
50,KPIGREEN,497.21,731.64,26.70,H-SC,13.38,129679.0,4382.0,54699.0,-3.36,3.50,42.18,47.15,144.0,0.08,0.99,62.24,MH,ATH,POWER
76,TANLA,917.30,1963.11,-30.49,H-SC,7.61,170172.0,-69243.0,342199.0,-2.99,-28.92,201.09,114.01,133.0,-0.20,1.30,56.07,AR,ATH,IT
74,SURYODAY,189.78,240.00,40.35,H-SC,5.06,99540.0,-50386.0,90064.0,-2.88,-33.61,90.48,26.46,142.0,-0.56,0.76,27.52,XR,NTT,BANKS


In [14]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,INDIGOPNTS,1407.73,1408.0,125.45,H-SC,4.70,147746.0,-26813.0,26845.0,-1.12,-15.36,18.17,0.02,135.0,-1.00,1.13,28.12,OX40N,NTT,PAINTS
82,TTKPRESTIG,769.29,770.0,-9.57,H-SC,11.65,82766.0,-18011.0,18101.0,0.51,-17.87,21.87,0.09,145.0,-1.00,0.63,8.74,OX40N,NTT,DURABLES
83,UJJIVANSFB,52.77,53.0,34.19,H-SC,20.64,116397.0,-26082.0,26701.0,-2.00,-18.31,22.94,0.44,163.0,-0.98,0.89,37.38,OX40N,NTT,BANKS
77,TATAELXSI,7332.28,7332.0,-22.02,H-MC,2.05,81844.0,-20808.0,20805.0,-1.94,-20.27,25.42,-0.00,101.0,-1.00,0.62,24.41,OX40N,NTT,IT
49,KANSAINER,299.63,340.0,-69.48,H-SC,1.66,212229.0,-57438.0,93763.0,-2.63,-21.30,44.18,13.47,136.0,-0.61,1.62,7.34,XY24,NTT,PAINTS


In [15]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
79,TCS,3794.03,4998.00,-27.38,H-LC,14.88,236340.0,-59594.0,153503.0,-1.11,-20.14,64.95,31.73,1.0,-0.39,1.80,0.90,X40,BTT,IT
88,VBL,492.64,671.64,-11.37,H-LC,11.34,268973.0,3933.0,92365.0,-1.60,1.48,34.34,36.33,3.0,0.04,2.05,14.98,X40N,ATH,FMCG
45,INFY,1461.46,2275.00,-19.60,H-LC,8.68,257563.0,-4038.0,149670.0,-1.47,-1.54,58.11,55.67,4.0,-0.03,1.97,4.47,X40,BTT,IT
1,ABB,5551.76,7934.00,-44.83,H-LC,8.78,196794.0,-19725.0,112625.0,-0.75,-9.11,57.23,42.91,5.0,-0.18,1.50,2.86,AR,NTT,ELECTRICAL
34,HCLTECH,1562.67,1943.91,0.53,H-LC,9.09,178120.0,-12526.0,59029.0,-1.53,-6.57,33.14,24.40,8.0,-0.21,1.36,8.32,X40,ATH,IT


In [16]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,MEDANTA,1087.93,1486.0,19.2,H-SC,13.41,152876.0,26676.0,19507.0,-0.67,21.14,12.76,36.59,156.0,1.37,1.17,36.3,XY24,NTT,HEALTHCARE


In [17]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,KPIGREEN,497.21,731.64,26.70,H-SC,13.38,129679.0,4382.0,54699.0,-3.36,3.50,42.18,47.15,144.0,0.08,0.99,62.24,MH,ATH,POWER
55,MEDANTA,1087.93,1486.00,19.20,H-SC,13.41,152876.0,26676.0,19507.0,-0.67,21.14,12.76,36.59,156.0,1.37,1.17,36.30,XY24,NTT,HEALTHCARE
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.92,157098.0,6185.0,51120.0,-1.28,4.10,32.54,37.97,90.0,0.12,1.20,30.44,AR,ATH,AUTO
19,CAMS,3643.00,5226.82,-5.69,H-SC,1.80,105834.0,3830.0,40513.0,-0.44,3.75,38.28,43.48,120.0,0.09,0.81,21.16,X40N,ATH,MISC
25,DABUR,505.20,735.00,-6.64,H-MC,4.99,202312.0,5284.0,84344.0,-1.13,2.68,41.69,45.49,100.0,0.06,1.54,16.07,XY24,BTT,FMCG


In [18]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HEROMOTOCO,4311.81,5949.07,-2.38,H-MC,4.92,157098.0,6185.0,51120.0,-1.28,4.10,32.54,37.97,90.0,0.12,1.20,30.44,AR,ATH,AUTO
90,WHIRLPOOL,1219.98,2270.00,-41.46,M-SC,5.72,98490.0,6992.0,71760.0,-0.30,7.64,72.86,86.07,211.0,0.10,0.75,42.48,XR,NTT,DURABLES
50,KPIGREEN,497.21,731.64,26.70,H-SC,13.38,129679.0,4382.0,54699.0,-3.36,3.50,42.18,47.15,144.0,0.08,0.99,62.24,MH,ATH,POWER
42,INDIAMART,2327.09,4850.92,-48.64,H-SC,13.39,133491.0,10155.0,123613.0,1.28,8.23,92.60,108.45,118.0,0.08,1.02,32.43,AR,ATH,MISC
91,WIPRO,243.46,420.00,-12.85,M-LC,5.91,149327.0,-1618.0,111069.0,-2.06,-1.07,74.38,72.51,55.0,-0.01,1.14,4.89,XR,NTT,IT


In [19]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
23,COFFEEDAY,59.14,80.00,-55.73,L-SC,21.58,66163.0,-47386.0,87434.0,-2.19,-41.73,132.15,35.27,267.0,-0.54,0.50,61.94,XR,NTT,HOTELS
54,MASFIN,326.60,399.50,-20.05,H-SC,16.30,91065.0,-6915.0,28786.0,0.50,-7.06,31.61,22.32,151.0,-0.24,0.69,32.08,XR,ATH,FINANCE
3,ALKYLAMINE,2347.98,4546.37,0.03,H-SC,18.81,93104.0,-7859.0,102386.0,-2.20,-7.78,109.97,93.63,149.0,-0.08,0.71,38.68,SR,ATH,CHEMICALS
36,HINDALCO,651.95,761.55,-19.90,H-LC,5.30,110136.0,5824.0,11707.0,0.18,5.58,10.63,16.81,10.0,0.50,0.84,22.53,X5K,ATH,METALS
83,UJJIVANSFB,52.77,53.00,34.19,H-SC,20.64,116397.0,-26082.0,26701.0,-2.00,-18.31,22.94,0.44,163.0,-0.98,0.89,37.38,OX40N,NTT,BANKS


In [20]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LALPATHLAB,2838.53,3545.00,-23.82,H-SC,7.02,216628.0,23608.0,24436.0,-0.08,12.23,11.28,24.89,128.0,0.97,1.65,38.43,X40N,NTT,HEALTHCARE
55,MEDANTA,1087.93,1486.00,19.20,H-SC,13.41,152876.0,26676.0,19507.0,-0.67,21.14,12.76,36.59,156.0,1.37,1.17,36.30,XY24,NTT,HEALTHCARE
69,SHALBY,261.39,327.00,935.09,M-SC,8.42,140679.0,-40464.0,85927.0,-1.85,-22.34,61.08,25.10,233.0,-0.47,1.07,12.74,XY24,NTT,HEALTHCARE
88,VBL,492.64,671.64,-11.37,H-LC,11.34,268973.0,3933.0,92365.0,-1.60,1.48,34.34,36.33,3.0,0.04,2.05,14.98,X40N,ATH,FMCG
27,DIXON,14367.00,18940.15,-28.24,H-MC,6.61,132152.0,17216.0,19373.0,-1.96,14.98,14.66,31.83,85.0,0.89,1.01,48.73,X40N,ATH,IT


In [21]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.71
1,25,41.57
2,50,71.48


In [22]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    40.60
LC    37.22
MC    22.20
Name: CurrAlloc%, dtype: float64

In [23]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     28.93
X40N     15.20
X40      13.30
XY25     11.76
XR        8.96
AR        7.68
OX40N     7.67
X5K       2.23
MH        1.75
X200      1.39
SR        1.15
Name: CurrAlloc%, dtype: float64

In [24]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.09
H-SC    27.32
H-MC    19.23
M-SC    11.87
M-LC     7.07
M-MC     2.62
L-SC     1.41
L-LC     1.06
L-MC     0.35
Name: CurrAlloc%, dtype: float64

In [25]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.32,-2.76,35.80
IT,12.01,-18.44,75.97
FINANCE,8.35,-19.12,62.35
BANKS,7.48,-16.31,63.77
PAINTS,5.78,-15.28,38.00
MISC,5.12,-21.80,93.06
HEALTHCARE,5.10,1.83,22.96
ELECTRICAL,4.70,-14.02,63.40
AUTO,4.39,-17.52,68.64


In [26]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2844529.0,23
XR,1117560.0,13
AR,1058218.0,9
X40,831980.0,10
X40N,616948.0,12
XY25,510189.0,8
OX40N,372478.0,10
SR,192601.0,2
X5K,147634.0,2


In [27]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2741809.0,30
M-SC,1413730.0,15
H-LC,1343865.0,20
H-MC,1188385.0,15
M-LC,452755.0,5
M-MC,334639.0,2
L-SC,275969.0,3
L-MC,57118.0,1
L-LC,48364.0,1


In [28]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      951798.0      7
M-SC       XY24      863362.0      7
H-SC       AR        612639.0      4
H-LC       X40       552141.0      6
H-SC       XR        519833.0      6
H-MC       XY24      463129.0      4
M-MC       XY24      334639.0      2
H-LC       X40N      313999.0      6
           AR        234745.0      2
H-SC       X40N      219072.0      4
H-MC       XY25      211883.0      2
           X40       203759.0      3
H-SC       SR        192601.0      2
           OX40N     191167.0      6
M-LC       XY24      189290.0      2
L-SC       XR        186768.0      2
M-SC       AR        159714.0      2
M-LC       X5K       135927.0      1
M-SC       XY25      122283.0      1
H-MC       XR        121786.0      1
M-SC       XR        120986.0      2
H-LC       XY25      111190.0      3
M-LC       XR        111069.0      1
L-SC       OX40N      89201.0      1
H-MC       X40N       83877.0      2
H-LC       X200       77772.0      1
M-SC       X40        76080.0      1
           OX40N      71305.0      2
L-MC       XR         57118.0      1
H-SC       MH         54699.0      1
H-MC       AR         51120.0      1
L-LC       XY25       48364.0      1
H-LC       XY24       42311.0      1
H-MC       MH         32026.0      1
           OX40N      20805.0      1
M-LC       XY25       16469.0      1
H-LC       X5K        11707.0      1

In [29]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
